In [1]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [2]:
sqlContext = SQLContext(sc)
df = sqlContext.read.load('/FileStore/tables/rzx8qo971493648385233/minute_weather_csv-aac7b.gz',format='com.databricks.spark.csv',header='true',inferSchema='true')

In [3]:
df.count()

In [4]:
filteredDF = df.filter((df.rowID%10)==0)
filteredDF.count()

In [5]:
filteredDF.describe().toPandas().transpose()

In [6]:
workingDF = filteredDF.drop('rain_accumulation').drop('rain_duration')

In [7]:
workingDF.describe().toPandas().transpose()

In [8]:
workingDF.count()
workingDF = workingDF.na.drop()
workingDF.count()
workingDF.columns

In [9]:
featuresUsed = ['air_pressure','air_temp','avg_wind_direction',
 'avg_wind_speed',
 'max_wind_direction',
 'max_wind_speed',
 'min_wind_direction',
 'min_wind_speed',
 'relative_humidity']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="unscaledFeatures")
assembled = assembler.transform(workingDF)

In [10]:
scaler = StandardScaler(inputCol="unscaledFeatures",outputCol="features",withStd=True,withMean=True)
scalerModel = scaler.fit(assembled)
scalerData = scalerModel.transform(assembled)

In [11]:
scaledDataFeat = scalerData.select("features")
scaledDataFeat.persist()

In [12]:
kmeans = KMeans(k=12, seed=1)
model =  kmeans.fit(scaledDataFeat)
tranformed = model.transform(scaledDataFeat)

In [13]:
centers = model.clusterCenters()
centers